In [1]:
import numpy as np
#import tensorflow as tf
import tensorflow.compat.v1 as tf
import pandas as pd
import time
from datetime import datetime

In [2]:
def f(x):
    diff = x - df1.Date.min()
    return diff

days = df1['Date'].apply(f)
d = pd.DataFrame(days)
d.rename(columns = {'Date':'Days'}, inplace = True)
d


NameError: name 'df1' is not defined

In [4]:
merge = pd.merge(df1, d, left_index = True, right_index = True)
merge

,Date,Time,Lat,Lon,Depth,Mag,Magt,Days
0,1900-10-10,00:28:17.600000,57.090,-153.480,0.0,8.3,ML,0 days
1,1901-01-18,00:39:00,60.000,-135.000,0.0,7.1,ML,100 days
2,1901-12-30,00:34:00,52.000,-160.000,0.0,6.6,ML,446 days
3,1901-12-31,00:02:43.100000,51.450,-171.020,0.0,7.8,ML,447 days
4,1902-01-01,00:20:14.800000,52.380,-167.450,0.0,7.8,ML,448 days
...,...,...,...,...,...,...,...,...
363065,2012-12-31,00:00:16.200000,16.625,-98.755,35.0,4.0,Mb,40990 days
363066,2012-12-31,00:11:59,52.788,171.431,48.4,4.5,Mb,40990 days
363067,2012-12-31,00:33:53.900000,14.083,-92.184,62.4,4.4,Mb,40990 days
363068,2012-12-31,00:02:04.900000,-61.596,154.288,10.0,5.2,Mb,40990 days


In [5]:
merge.to_csv('Merge.csv')

In [3]:
df = pd.read_csv('C:/Users/stefa/Downloads/Merge.csv')
df

,Date,Time,Lat,Lon,Depth,Mag,Magt,Days
0,1900-10-10,28:17.6,57.090,-153.480,0.0,8.3,ML,0
1,1901-01-18,00:39:00,60.000,-135.000,0.0,7.1,ML,100
2,1901-12-30,00:34:00,52.000,-160.000,0.0,6.6,ML,446
3,1901-12-31,02:43.1,51.450,-171.020,0.0,7.8,ML,447
4,1902-01-01,20:14.8,52.380,-167.450,0.0,7.8,ML,448
...,...,...,...,...,...,...,...,...
363065,2012-12-31,00:16.2,16.625,-98.755,35.0,4.0,Mb,40990
363066,2012-12-31,00:11:59,52.788,171.431,48.4,4.5,Mb,40990
363067,2012-12-31,33:53.9,14.083,-92.184,62.4,4.4,Mb,40990
363068,2012-12-31,02:04.9,-61.596,154.288,10.0,5.2,Mb,40990


In [4]:
col1 = df[['Days','Depth']]
col2 = df['Mag']
#Convert to Numpy array
InputX1 = col1.to_numpy()
InputY1 = col2.to_numpy()
print(InputX1)

[[0.000e+00 0.000e+00]
 [1.000e+02 0.000e+00]
 [4.460e+02 0.000e+00]
 ...
 [4.099e+04 6.240e+01]
 [4.099e+04 1.000e+01]
 [4.099e+04 5.280e+01]]


In [5]:
#Min-max Normalization
X1_min = np.amin(InputX1,0)     
X1_max = np.amax(InputX1,0)   
print("Mininum values:",X1_min)
print("Maximum values:",X1_max)
Y1_min = np.amin(InputY1)     
Y1_max = np.amax(InputY1) 
InputX1_norm = (InputX1-X1_min)/(X1_max-X1_min)
InputY1_norm = InputY1  #No normalization in output

#Reshape
Xfeatures = 2 #Number of input features
Yfeatures = 1 #Number of input features
samples = 23000 # Number of samples

InputX1_reshape = np.resize(InputX1_norm,(samples,Xfeatures))
InputY1_reshape = np.resize(InputY1_norm,(samples,Yfeatures))

Mininum values: [0. 0.]
Maximum values: [40990.    735.8]


In [6]:
#Training data
batch_size = 2000
InputX1train = InputX1_reshape[0:batch_size,:]
InputY1train = InputY1_reshape[0:batch_size,:]
#Validation data
v_size = 2500
InputX1v = InputX1_reshape[batch_size:batch_size+v_size,:]
InputY1v = InputY1_reshape[batch_size:batch_size+v_size,:]

In [7]:
learning_rate = 0.001
training_iterations = 1000
display_iterations = 200

In [8]:
tf.compat.v1.disable_eager_execution()
#Input
X = tf.compat.v1.placeholder(tf.float32,shape=(None,Xfeatures))
#tf.compat.v1.placeholder(shape=[None, 2], dtype=tf.float32)
#Output
Y = tf.compat.v1.placeholder(tf.float32)


In [9]:
#Neurons
L1 = 3
L2 = 3
L3 = 3

#Layer1 weights
W_fc1 = tf.Variable(tf.random_uniform([Xfeatures,L1]))
b_fc1 = tf.Variable(tf.constant(0.1,shape=[L1]))

#Layer2 weights
W_fc2 = tf.Variable(tf.random_uniform([L1,L2]))
b_fc2 = tf.Variable(tf.constant(0.1,shape=[L2]))

#Layer3 weights
W_fc3 = tf.Variable(tf.random_uniform([L2,L3]))
b_fc3 = tf.Variable(tf.constant(0.1,shape=[L3]))

#Output layer weights
W_fO= tf.Variable(tf.random_uniform([L3,Yfeatures]))
b_fO = tf.Variable(tf.constant(0.1,shape=[Yfeatures]))

In [10]:
#Layer 1
matmul_fc1=tf.matmul(X, W_fc1) + b_fc1
h_fc1 = tf.nn.relu(matmul_fc1)   #ReLU activation
#Layer 2
matmul_fc2=tf.matmul(h_fc1, W_fc2) + b_fc2
h_fc2 = tf.nn.relu(matmul_fc2)   #ReLU activation
#Layer 3
matmul_fc3=tf.matmul(h_fc2, W_fc3) + b_fc3
h_fc3 = tf.nn.relu(matmul_fc3)   #ReLU activation
#Output layer
matmul_fc4=tf.matmul(h_fc3, W_fO) + b_fO
output_layer = matmul_fc4  #linear activation

In [11]:
#Loss function
mean_square =  tf.reduce_mean(tf.square(Y-output_layer))
train_step = tf.train.AdamOptimizer(learning_rate).minimize(mean_square)

#Operation to save variables
saver = tf.train.Saver()

In [12]:
#Initialization and session
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    print("Training loss:",sess.run([mean_square],feed_dict={X:InputX1train,Y:InputY1train}))
    for i in range(training_iterations):
        sess.run([train_step],feed_dict={X:InputX1train,Y:InputY1train})
        if i%display_iterations ==0:
            print("Training loss is:",sess.run([mean_square],feed_dict={X:InputX1train,Y:InputY1train}),"at itertion:",i)
            print("Validation loss is:",sess.run([mean_square],feed_dict={X:InputX1v,Y:InputY1v}),"at itertion:",i)
    # Save the variables to disk.
    save_path = saver.save(sess, "/tmp/earthquake_model.ckpt")
    print("Model saved in file: %s" % save_path)

    print("Final training loss:",sess.run([mean_square],feed_dict={X:InputX1train,Y:InputY1train}))
    print("Final validation loss:",sess.run([mean_square],feed_dict={X:InputX1v,Y:InputY1v}))

Training loss: [14.145834]
Training loss is: [14.039497] at itertion: 0
Validation loss is: [10.592797] at itertion: 0
Training loss is: [1.4680523] at itertion: 200
Validation loss is: [0.9156082] at itertion: 200
Training loss is: [1.302198] at itertion: 400
Validation loss is: [0.76631737] at itertion: 400
Training loss is: [1.195806] at itertion: 600
Validation loss is: [0.6339214] at itertion: 600
Training loss is: [1.1227071] at itertion: 800
Validation loss is: [0.55096775] at itertion: 800
Model saved in file: /tmp/earthquake_model.ckpt
Final training loss: [1.0691878]
Final validation loss: [0.49770713]


In [18]:
from datetime import date
d0 = date(1900, 1, 1)
d1 = date(2012, 4, 26)
delta = d1 - d0
print(delta.days)

41023


In [22]:
#Testing
lat = input("Enter Latitude between -90 to 90:")
long = input("Enter Longitude between -180 to 180:")
depth = input("Enter Depth between 0 to 800:")
days = input("Enter the number of days:")
InputX2 = np.asarray([[lat,long,depth,days]],dtype=np.float32)
InputX2_norm = (InputX2-X1_min)/(X1_max-X1_min)
InputX1test = np.resize(InputX2_norm,(1,Xfeatures))
with tf.Session() as sess:
    # Restore variables from disk for validation.
    saver.restore(sess, "/tmp/earthquake_model.ckpt")
    print("Model restored.")
    #print("Final validation loss:",sess.run([mean_square],feed_dict={X:InputX1v,Y:InputY1v}))
    print("output:",sess.run([output_layer],feed_dict={X:InputX1test}))

Enter Latitude between -90 to 90:10
Enter Longitude between -180 to 180:100
Enter Depth between 0 to 800:100
Enter the number of days:41000
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
Model restored.
output: [array([[5.3156357]], dtype=float32)]
